In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
%matplotlib inline

In [3]:
anime = pd.read_csv("anime-recommendations-database/anime.csv")

In [4]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


# Clean up and Adding stuff manually

In [25]:
anime.loc[(anime["type"] == "Movie") & (anime["episodes"] == "Unknown")] = "1" # Movies are typically 1 'episode'
anime.loc[(anime["genre"]=="Hentai") & (anime["episodes"]=="Unknown"),"episodes"] = "1"
anime.loc[(anime["type"]=="OVA") & (anime["episodes"]=="Unknown"),"episodes"] = "1"
anime["members"] = anime["members"].astype(float)

In [14]:
preknown = {"Naruto Shippuuden":500, "One Piece":784,"Detective Conan":854, "Dragon Ball Super":86,
                "Crayon Shin chan":942, "Yu Gi Oh Arc V":148,"Shingeki no Kyojin Season 2":25,
                "Boku no Hero Academia 2nd Season":25,"Little Witch Academia TV":25} # Got this from the internet 

In [15]:
for i,j in preknown.items():    
    anime.loc[anime["name"]==i,"episodes"] = j

In [16]:
anime["episodes"] = anime["episodes"].map(lambda x:np.nan if x=="Unknown" else x)

## Filling up Unknown Values in Rating

In [17]:
anime["rating"] = anime["rating"].astype(float)
anime["rating"].fillna(anime["rating"].median(),inplace = True) # Filled Unknown Values with median of all values 

In [18]:
pd.get_dummies(anime[["type"]]).head() # Convert type to categorical values for ease

,type_1,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [26]:
## Using MaxAbsScaler as it preserves the sparsity while scaing the values from 0-1.

anime_features = pd.concat([anime["genre"].str.get_dummies(sep=","),pd.get_dummies(anime[["type"]]),
                            anime[["rating"]],anime[["members"]],anime["episodes"]],axis=1)

anime["name"] = anime["name"].map(lambda name:re.sub('[^A-Za-z0-9]+', " ", name))

In [27]:
anime_features.head()

from sklearn.preprocessing import MaxAbsScaler

max_scaler = MaxAbsScaler()
anime_features = max_scaler.fit_transform(anime_features)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').